Imports

In [2]:
# my imports
import requests
from lxml import html
import pandas as pd
from bs4 import BeautifulSoup
import re
import time
import urllib

Code Fragements (Skip down)


In [ ]:
# session = requests.Session()
# page = requests.get('https://www.lineups.com/nfl/player-stats/matt-ryan')
# # print(page)
# soup = BeautifulSoup(page, 'lxml')
# print(soup)

# from lxml.html.clean import clean_html
# from selenium import webdriver
# from selenium.webdriver.support.ui import Select
# from selenium.webdriver.common.keys import Keys

# # the location of a webdriver on my computer
# # this thing is probably what I need if I want to extract from complex websites where forms, and buttons, and parameters
# # need to be changed on the website.
# driver = webdriver.Chrome('C:/NEW PROGRAMS/chromedriver_win32/chromedriver.exe')

# driver.get("https://www.lineups.com/nfl/roster/new-england-patriots")
# # driver.get("https://www.lineups.com/nfl/player-stats/Matt-Ryan")

#     time.sleep()

#     filename = "nfl_ne_roster.html"
#     soupString = open(filename, "r").read()
#     soup = BeautifulSoup(soupString, "html.parser")

# pos_df = pd.DataFrame(position_data)
# pos_df.fillna("Rookie", inplace=True)
# pos_df.columns = position_headers

# # filename = position + '-stats.xlsx'.xlsx
# print(filename)
# pos_df.to_excel(filename)

# position_data
# for each in position_data:
#     if len(each)>9:
#         print (each)
# print(df.loc[df['Name'] == 'John Franklin'])

# # with open(filename, "w") as file:
# #     file.write(str(soup))

# soupString = open(filename, "r").read()

# # turn back into BeautifulSoup
# soup = BeautifulSoup(soupString, "html.parser")
# pos_unique = list(df['Pos'].unique())
# print(type(pos_unique[14]))
# pos_df = df['Pos'] == 'OL'
# pos_df = df[pos_df]
# print(len(pos_df))
# print(pos_df)


Input the text file nflTeams.txt that is a list of all 32 NFL teams

Then, format the textfile into a list of properly formed URL endings

In [168]:
file = open("nflTeams.txt", "r")
nflTeams = file.read().splitlines()

for i, team in enumerate(nflTeams):
    nflTeams[i] = nflTeams[i].replace(" ", "-").lower()

print(nflTeams[:5])

['arizona-cardinals', 'atlanta-falcons', 'baltimore-ravens', 'buffalo-bills', 'carolina-panthers']


Get roster data from all NFL teams: https://www.lineups.com/nfl/roster/

Then export it to an excel file

Total NFL players

Get the headers for the Roster Data

And print

Create dataframe of data and export to excel file named:

    'nfl_rosters.xlsx'

In [31]:
# df = pd.DataFrame(teamData, columns = headerColList)
# df.to_excel('nfl_rosters.xlsx')

<h2>Getting the Positional Stats of Each Player</h2>

Import the made excel sheet from previous code into notebook

(So you don't have to rescrape every time)

In [193]:
filename = "nfl_rosters.xlsx"
df = pd.read_excel(filename)
# print(df.tail)


Check all the positions that I am making a separate table for

In [191]:
pos_unique = list(df['Pos'].unique())
print(pos_unique)


['C', 'CB', 'DB', 'DE', 'DL', 'DT', 'FB', 'FS', 'G', 'ILB', 'K', 'LB', 'LS', 'NT', 'OL', 'OLB', 'OT', 'P', 'QB', 'RB', 'S', 'SS', 'TE', 'WR', nan]


Get all the player stats and export it to an excel file